In [81]:
import cmlreaders as cml
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import warnings; warnings.simplefilter('ignore')
pd.options.display.max_columns=None
pd.options.display.max_rows=100
from IPython.display import display, Markdown, HTML
import pandas_to_pybeh as pb
import time
import rep_pybeh as rpb
from tqdm import tqdm_notebook as lol
import glob
from plotting import *

In [82]:
evs = []
data = cml.get_data_index(kind = 'r1'); data = data[data['experiment'] == 'RepFR1']
for subject, df in data.groupby('subject'):
    for session in pd.unique(df['session']):
        r = cml.CMLReader(subject=subject, experiment='RepFR1', session=session)
        temp = r.load('task_events')
        evs.append(temp)
evs = pd.concat(evs)
evs = evs[evs.list != 0]
evs.subject.unique()

array(['R1204T', 'R1501J', 'R1514E', 'R1516E', 'R1528E', 'R1531T',
       'R1534D', 'R1547D', 'R1556J', 'R1564J', 'R1566D', 'R1568E',
       'R1579T', 'R1582E', 'R1584J', 'R1586T', 'R1587J', 'R1589T',
       'R1590T', 'R1594E', 'R1596T'], dtype=object)

In [83]:
beh = evs[evs.subject == 'R1596T']
beh = beh[beh.session == 0]
evs = rpb.get_act_spos(beh)
rpb.temp_fact(evs, 27)

0.5409596530920056

In [69]:
actual = []
list_length = 27
recs = evs[evs.type == 'REC_WORD']
for l, df in recs.groupby('list'):
    used_positions = np.zeros(list_length)
    serialpos = df.act_serialpos.to_numpy()
    crp = pd.DataFrame(columns=['lag', 'prob'])
    crp['lag'] = pd.Series(range(-list_length +1, list_length))
    for i in range(len(serialpos)-1):
        try:
            for j, previous in enumerate(serialpos[i]):
                for k, current in enumerate(serialpos[i+1]):
                    lag = abs(current - previous)
                    actual.append(lag)
            total = 0
            count = 0
            open_pos = np.where(used_positions==0)
            possible = []
            for j, position in enumerate(serialpos[i]):
                possible = abs(open_pos - position)[0]
            for lag in actual:
                rank = temp_percentile_rank(lag, possible)
                if rank is not None:
                    total += rank
                    count += 1
        except Exception as e:
            pass
if count == 0:
            count= np.nan
temp_fact = total / count

0.628597592883306


In [71]:
def temp_fact(evs, list_length): 
    actual = []
    recs = evs[evs.type == 'REC_WORD']
    for l, df in recs.groupby('list'):
        used_positions = np.zeros(list_length)
        serialpos = df.act_serialpos.to_numpy()
        crp = pd.DataFrame(columns=['lag', 'prob'])
        crp['lag'] = pd.Series(range(-list_length +1, list_length))
        for i in range(len(serialpos)-1):
            try:

                for j, previous in enumerate(serialpos[i]):

                    for k, current in enumerate(serialpos[i+1]):
                        lag = abs(current - previous)
                        actual.append(lag)

                total = 0
                count = 0
                open_pos = np.where(used_positions==0)
                possible = []
                for j, position in enumerate(serialpos[i]):
                    possible = abs(open_pos - position)[0]
                for lag in actual:
                    rank = temp_percentile_rank(lag, possible)
                    if rank is not None:
                        total += rank
                        count += 1
            except Exception as e:
                pass

    if count == 0:
                count= np.nan
    temp_fact = total / count
    return temp_fact

In [65]:
def temp_percentile_rank(actual, possible):
    """
    Helper function to return the percentile rank of the actual transition within the list of possible transitions.

    :param actual: The distance of the actual transition that was made.
    :param possible: The list of all possible transition distances that could have been made.

    :return: The proportion of possible transitions that were more distant than the actual transition.
    """
    # If there were fewer than 2 possible transitions, we can't compute a meaningful percentile rank
    if len(possible) < 2:
        return None

    # Sort possible transitions from largest to smallest
    possible = sorted(possible)[::-1]

    # Get indices of the one or more possible transitions with the same distance as the actual transition
    matches, = np.where(possible == actual)

    if len(matches) > 0:
        # Get the number of possible transitions that were more distant than the actual transition
        # If there were multiple transitions with the same distance as the actual one, average across their ranks
        rank = np.mean(matches)
        # Convert rank to the proportion of possible transitions that were more distant than the actual transition
        ptile_rank = rank / (len(possible) - 1.)
    else:
        ptile_rank = None

    return ptile_rank